# pac preparation

Run this notebook to import new calibration scans. Shouldn't need any particular configuration or changes; it just globs for plausible-looking cal files. Runs pretty quickly.

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random
import pickle
from os.path import join

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [17]:
cals = list()
for d in ["data/raw/GBT", "data/raw/AO", "data/raw/AO/from_UBC", "data/raw/GBT/new/"]:
    cals.extend(glob(join(d,"*uppi_*_cal_*.*")))
    cals.extend(glob(join(d,"*.cf")))
    cals.extend(glob(join(d,"*.cz")))
for d in ["data/cal/fluxcal/AO", "data/cal/fluxcal/AO/new", "data/cal/fluxcal/GBT"]:
    cals.extend(glob(join(d,"*.fcal")))
cals.extend(glob(join("data","cal","*.cf")))
print len(cals)
cals = [c for c in set(cals) if not c.endswith(".bad")]
cals.sort()
print len(cals)

with open("data/cal/cal-list.ls", "wt") as f:
    for c in cals:
        print c
        f.write(c)
        f.write("\n")

818
692
data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5481_cal.fcal
data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5485_cal.fcal
data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5874_cal.fcal
data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5878_cal.fcal
data/cal/fluxcal/AO/new/puppi_57502_J1445+099cal_0013_cal.fcal
data/cal/fluxcal/AO/new/puppi_57502_J1445+099cal_0017_cal.fcal
data/cal/fluxcal/AO/new/puppi_57523_J1445+099cal_0113_cal.fcal
data/cal/fluxcal/AO/new/puppi_57523_J1445+099cal_0117_cal.fcal
data/cal/fluxcal/AO/new/puppi_57537_J1445+099cal_0575_cal.fcal
data/cal/fluxcal/AO/new/puppi_57537_J1445+099cal_0579_cal.fcal
data/cal/fluxcal/AO/new/puppi_57564_J1445+099cal_0437_cal.fcal
data/cal/fluxcal/AO/new/puppi_57564_J1445+099cal_0441_cal.fcal
data/cal/fluxcal/AO/new/puppi_57586_J1445+099cal_0277_cal.fcal
data/cal/fluxcal/AO/new/puppi_57586_J1445+099cal_0281_cal.fcal
data/cal/fluxcal/AO/new/puppi_57606_J1445+099cal_0563_cal.fcal
data/cal/fluxcal/AO/new/puppi_57606_J1445+099ca

In [18]:
P = subprocess.Popen(["pac",
                 "-w",
                 "-k", "data/cal/cal-new.db",
                 "-W", "data/cal/cal-list.ls"], 
                 stderr=subprocess.PIPE, 
                 stdout=subprocess.PIPE)
out, err = P.communicate()
print out
print err
if P.returncode:
    raise ValueError("Return code %d" % P.returncode)

pac: Generating new calibrator database
pac: Loading calibrator filenames from metafile=data/cal/cal-list.ls
pac: Writing database summary file to data/cal/cal-new.db

Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5481_cal.fcal and
	data/cal/fluxcal/AO/puppi_57389_J1445+099cal_5481_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57389_J1445+099cal_5485_cal.fcal and
	data/cal/fluxcal/AO/puppi_57389_J1445+099cal_5485_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5874_cal.fcal and
	data/cal/fluxcal/AO/puppi_57417_J1445+099cal_5874_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/puppi_57417_J1445+099cal_5878_cal.fcal and
	data/cal/fluxcal/AO/puppi_57417_J1445+099cal_5878_cal.fcal
Pulsar::Database::add keeping newest of duplicate entries:
	data/cal/fluxcal/AO/new/

In [19]:
# Okay I trust the new cal DB
shutil.copy("data/cal/cal-new.db", "data/cal/cal.db")
